\title{Measuring line geographical features} \author{} \maketitle

# Table of Contents
 <p><div class="lev1 toc-item"><a href="#The-data-source" data-toc-modified-id="The-data-source-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>The data source</a></div><div class="lev1 toc-item"><a href="#other-requeired-tool" data-toc-modified-id="other-requeired-tool-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>other requeired tool</a></div><div class="lev1 toc-item"><a href="#The-codes" data-toc-modified-id="The-codes-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>The codes</a></div>

# The data source
1. Open street map shape files

# other requeired tool
1. Anaconda Python distributuion
1. Jupyter notebook
1. Google aearth
1. Qgis

# The codes

In [ ]:
import numpy as np
import pandas as pd
from geopandas import GeoDataFrame
from shapely.geometry import MultiPoint, MultiPolygon, line
import geopandas as gp
import ast
import fiona
from shapely.geometry import shape, mapping,LineString
from shapely.ops import cascaded_union, unary_union
import fiona
from shapely.geometry import shape
from rtree import index


bufSHP = 'Emis_inv_grid_v1.shp'
intSHP = 'Roadways_gridV6.shp'
#the new shape file going to be created
ctSHP  = 'Roadways_v3.shp'

with fiona.open(bufSHP, 'r') as layer1:
    with fiona.open(ctSHP, 'r') as layer2:
        # We copy schema and add the  new property for the new resulting shp
        schema = layer2.schema.copy()
        schema['properties']['gid'] = 'int:10'
        # We open a first empty shp to write new content from both others shp
        with fiona.open(intSHP, 'w', 'ESRI Shapefile', schema) as layer3:
            index = rtree.index.Index()
            for feat1 in layer1:
                fid = int(feat1['id'])
                geom1 = shape(feat1['geometry'])
                index.insert(fid, geom1.bounds)
            for feat2 in layer2:
                geom2 = shape(feat2['geometry'])
                for fid in list(index.intersection(geom2.bounds)):
                    if fid != int(feat2['id']):
                        feat1 = layer1[fid]
                        geom1 = shape(feat1['geometry'])
                        if geom1.intersects(geom2):
                            # We take attributes from ctSHP
                            props = feat2['properties']
                            # Then append the uid attribute we want from the other shp
                            props['gid'] = feat1['properties']['gid']
                            geom3=geom1.intersection(geom2)
                            props['length']=geom3.length*100
                            # Add the content to the right schema in the new shp
                            layer3.write({
                                'properties': props,
                                'geometry': mapping(geom1.intersection(geom2))
                            })